In [2]:
from sqlalchemy import create_engine
import pandas as pd
import os

# Configuración de conexión a MySQL
mysql_host = "localhost"  # Cambia según tu configuración
mysql_port = 3306         # Puerto de MySQL
mysql_user = "root" # Usuario de MySQL
mysql_password = "root1234" # Contraseña de MySQL
mysql_db = "store"        # Nombre de la base de datos

# Crear el motor de conexión
engine = create_engine(f"mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_db}")

# Probar la conexión
try:
    with engine.connect() as connection:
        print("Conexión exitosa a MySQL")
except Exception as e:
    print(f"Error de conexión: {e}")

Conexión exitosa a MySQL


In [7]:
def cargar_csv_incremental(ruta_carpeta_csvs, nombre_tabla):
    """
    Lee archivos CSV de una carpeta y los carga incrementalmente en una tabla MySQL.

    :param ruta_carpeta_csvs: Ruta de la carpeta que contiene los archivos CSV.
    :param nombre_tabla: Nombre de la tabla MySQL donde se insertarán los datos.
    """
    # Iterar por los archivos CSV en la carpeta
    for archivo in sorted(os.listdir(ruta_carpeta_csvs)):
        if archivo.endswith(".csv"):
            ruta_csv = os.path.join(ruta_carpeta_csvs, archivo)
            print(f"Leyendo archivo: {ruta_csv}")
            
            # Leer el archivo CSV con pandas
            df = pd.read_csv(ruta_csv)

            # Limpiar los nombres de las columnas para eliminar caracteres especiales o espacios
            df.columns = df.columns.str.strip().str.replace('/', '_').str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

            # Cargar el DataFrame en la tabla MySQL
            try:
                # Insertar en la tabla (append para agregar registros)
                df.to_sql(nombre_tabla, con=engine, if_exists="append", index=False)
                print(f"Datos de {archivo} insertados exitosamente en la tabla {nombre_tabla}.")
            except Exception as e:
                print(f"Error al insertar datos de {archivo}: {e}")

# Llamar la función
ruta_carpeta_csvs = "Data/csv"  # Cambia esta ruta según tu configuración
nombre_tabla = "liquor_sales"  # Nombre de la tabla en MySQL
cargar_csv_incremental(ruta_carpeta_csvs, nombre_tabla)


Leyendo archivo: Data/csv\proyecto_2012-04-17.csv
Datos de proyecto_2012-04-17.csv insertados exitosamente en la tabla liquor_sales.
Leyendo archivo: Data/csv\proyecto_2012-05-02.csv
Error al insertar datos de proyecto_2012-05-02.csv: (pymysql.err.DataError) (1265, "Data truncated for column 'Zip_Code' at row 64")
[SQL: INSERT INTO liquor_sales (`Invoice_Item_Number`, `Date`, `Store_Number`, `Store_Name`, `Address`, `City`, `Zip_Code`, `Store_Location`, `County_Number`, `County`, `Category`, `Category_Name`, `Vendor_Number`, `Vendor_Name`, `Item_Number`, `Item_Description`, `Pack`, `Bottle_Volume_ml`, `State_Bottle_Cost`, `State_Bottle_Retail`, `Bottles_Sold`, `Sale_Dollars`, `Volume_Sold_Liters`, `Volume_Sold_Gallons`, sale_date) VALUES (%(Invoice_Item_Number)s, %(Date)s, %(Store_Number)s, %(Store_Name)s, %(Address)s, %(City)s, %(Zip_Code)s, %(Store_Location)s, %(County_Number)s, %(County)s, %(Category)s, %(Category_Name)s, %(Vendor_Number)s, %(Vendor_Name)s, %(Item_Number)s, %(Item_D